## Databrick on GCP BQ posting

This unsupported notebook notebook is supporting the posting [**Spark predicate pushdown for Google BigQuery**](https://medium.com/@frank.munz/predicate-pushdown-for-apache-spark-with-google-bigquery-2ad4f9e81e6).
### Write Spark DF to BQ dataset

In [0]:
from pyspark.sql.types import StringType

bucket = "databricks-bq-123"

# make sure this dataset exists, or create is e.g. using GCP console or CLI
table = "together.myTable"

mylist = ["Google", "Databricks", "better together"]
df = spark.createDataFrame(mylist, StringType())
df.write.format("bigquery")  \
    .option("temporaryGcsBucket", bucket) \
    .option("table", table) \
    .mode("overwrite") \
    .save()

### Python example BQ Storage API

In [0]:
table = "together.myTable"
spark.read.format("bigquery")   \
    .option("table", table)  \
    .load()  \
    .display()


value
better together
Databricks
Google


### Predicate Pushdown BQ Public Dataset with Explain

In [0]:
%scala 

// shorter example is working with predicate pushdown
val df = 
  spark.read.format("bigquery")
  .option("table", "bigquery-public-data.samples.natality")
  .load()
  .filter("state = 'CA'")
  .filter("weight_pounds > 11")

df.explain()


== Physical Plan ==
*(1) Scan BigQueryRelation(bigquery-public-data.samples.natality
numRows=137,826,763
numBytes=23,562,717,384
) [source_year#852L,year#853L,month#854L,day#855L,wday#856L,state#857,is_male#858,child_race#859L,weight_pounds#860,plurality#861L,apgar_1min#862L,apgar_5min#863L,mother_residence_state#864,mother_race#865L,mother_age#866L,gestation_weeks#867L,lmp#868,mother_married#869,mother_birth_state#870,cigarette_use#871,cigarettes_per_day#872L,alcohol_use#873,drinks_per_week#874L,weight_gain_pounds#875L,... 7 more fields] PushedFilters: [*IsNotNull(state), *IsNotNull(weight_pounds), *EqualTo(state,CA), *GreaterThan(weight_pounds,11.0)], ReadSchema: struct<source_year:bigint,year:bigint,month:bigint,day:bigint,wday:bigint,state:string,is_male:bo...


df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [source_year: bigint, year: bigint ... 29 more fields]

### BQ query with Storage API

In [0]:
%scala

val table = "bigquery-public-data.samples.shakespeare"
val tempLocation = "databricks_testing_frank"

// read the entire table into a DataFrame
val df1 = spark.read.format("bigquery").option("table", table).load()



table: String = bigquery-public-data.samples.shakespeare
tempLocation: String = databricks_testing_frank
df1: org.apache.spark.sql.DataFrame = [word: string, word_count: bigint ... 2 more fields]

### Shakespear Histo BQ Query

In [0]:
%scala

// public dataset 
val table = "bigquery-public-data.samples.shakespeare"

//val existing dataset where GCP user has table creation permission
val tempLocation = "mdataset"
// query string
val q = s"""SELECT word, SUM(word_count) AS word_count FROM ${table} 
        GROUP BY word ORDER BY word_count DESC LIMIT 10 """

// read the result of a BigQuery SQL query into a DataFrame
val df2 =
	spark.read.format("bigquery")
    .option("query", q)
    .option("materializationDataset", tempLocation)
	.load()

// show the top 5 common words in shakespeare
df2.show(5)
    

+----+----------+
word|word_count|
+----+----------+
 the| 25568|
 I| 21028|
 and| 19649|
 to| 17361|
 of| 16438|
+----+----------+
only showing top 5 rows

table: String = bigquery-public-data.samples.shakespeare
tempLocation: String = mdataset
q: String =
"SELECT word, SUM(word_count) AS word_count FROM bigquery-public-data.samples.shakespeare
 GROUP BY word ORDER BY word_count DESC LIMIT 10 "
df2: org.apache.spark.sql.DataFrame = [word: string, word_count: bigint]

In [0]:
table = "bigquery-public-data.samples.shakespeare"
df = spark.read.format("bigquery").option("table",table).load()
df.show()
df.createOrReplaceTempView("words")

+---------+----------+-------+-----------+
 word|word_count| corpus|corpus_date|
+---------+----------+-------+-----------+
 LVII| 1|sonnets| 0|
 augurs| 1|sonnets| 0|
 dimm'd| 1|sonnets| 0|
 plagues| 1|sonnets| 0|
 treason| 1|sonnets| 0|
 surmise| 1|sonnets| 0|
 heed| 1|sonnets| 0|
Unthrifty| 1|sonnets| 0|
 quality| 1|sonnets| 0|
 wherever| 1|sonnets| 0|
 C| 1|sonnets| 0|
 L| 1|sonnets| 0|
imaginary| 1|sonnets| 0|
 H| 1|sonnets| 0|
 relief| 1|sonnets| 0|
 W| 1|sonnets| 0|
 V| 1|sonnets| 0|
 advised| 1|sonnets| 0|
 grey| 1|sonnets| 0|
 X| 1|sonnets| 0|
+---------+----------+-------+-----------+
only showing top 20 rows

root
-- word: string (nullable = false)
-- word_count: long (nullable = false)
-- corpus: string (nullable = false)
-- corpus_date: long (nullable = false)

### Spark SQL: Shakespear Histo

In [0]:
# perform word count
wordCountDf = spark.sql("SELECT word, SUM(word_count) AS word_count FROM words GROUP BY word ORDER BY word_count DESC LIMIT 10")

display(wordCountDf)

word,word_count
the,25568
I,21028
and,19649
to,17361
of,16438
a,13409
you,12527
my,11291
in,10589
is,8735


### Shakespeare WordCount. SQL query executed on BQ with materializationDataset

In [0]:
table = "bigquery-public-data.samples.shakespeare"
tempLocation = "mdataset"
query = "SELECT count(1) FROM {table}".format(table=table)

# load the result of a SQL query on BigQuery into a DataFrame
df = spark.read.format("bigquery") \
.option("materializationDataset", tempLocation) \
.option("query", query) \
.load() \
.collect()

# display(df)

## Covid-19 Data

In [0]:


table = "bigquery-public-data.covid19_nyt.excess_deaths"
df = spark.read.format("bigquery").option("table",table).load()
df.createOrReplaceTempView("covid19_nyt_excess_deaths")
#df.display()



country,placename,frequency,start_date,end_date,year,month,week,deaths,expected_deaths,excess_deaths,baseline
Bolivia,null,monthly,null,null,2016,1,null,4538,null,null,null
Bolivia,null,monthly,null,null,2016,2,null,4244,null,null,null
Bolivia,null,monthly,null,null,2016,3,null,4312,null,null,null
Bolivia,null,monthly,null,null,2016,4,null,4134,null,null,null
Bolivia,null,monthly,null,null,2016,5,null,4442,null,null,null
Bolivia,null,monthly,null,null,2016,6,null,4720,null,null,null
Bolivia,null,monthly,null,null,2016,7,null,4377,null,null,null
Bolivia,null,monthly,null,null,2016,8,null,4296,null,null,null
Bolivia,null,monthly,null,null,2016,9,null,4119,null,null,null
Bolivia,null,monthly,null,null,2016,10,null,4210,null,null,null


In [0]:
%sql
select country, sum(excess_deaths) as excess_deaths from covid19_nyt_excess_deaths group by country


country,excess_deaths
Russia,23445
Sweden,7469
Turkey,15405
Germany,8072
France,45917
Belgium,14281
Ecuador,37479
Finland,1113
Peru,107023
India,8338
